In [3]:
import numpy as np
import networkx as nx

# Functions

In [4]:
def lattice2Dsq(x,y):
    '''
    Returns adjacency matrix of an x by y lattice graph on x*y nodes as a sparse matrix
    
    x (int): number of nodes in the x direction
    y (int): number of nodes in the y direction
    '''
    G = nx.grid_graph(dim=(x,y))
    G = nx.DiGraph(G)
    A = nx.to_scipy_sparse_array(G)
    A.setdiag(np.ones(x*y))
    return A 

def bf_clusters(num, pop):
    '''
    Returns an 1 by N array where the each index corresponds to a unit in the population and the value at that index is their cluster assignment

    Parameters
    ----------
    num (int)
        number of clusters (should be a perfect square nc*nc)
    pop (int)
        size of the population (should be a perfect square n*n)

    Returns
    -------
    clusters
        cluster assignments for each person
    
    clusters.flatten() (numpy array of size 1 by pop)
        cluster assignments for each person
        clusters[i]=j means that population unit i in [pop] is assigned to cluster j in [num]
    '''
    
    nc = int(np.sqrt(num)) #sqrt of the total number of clusters
    n = int(np.sqrt(pop))  #sqrt of the population size
    k = int(np.ceil(n/nc)) #"typical" cluster contains k*k units
    divides = n%k==0
    clusters = np.zeros((n,n), dtype=int)
    
    for i in range(n):
        for j in range(n):
            s,t = lat_toCluster(i,j,k,divides=divides) 
            clusters[i,j] = nc*s + t
    return clusters, clusters.flatten()

def lat_toCluster(I,J,k,q1=0,q2=0,divides = False):
    '''
    Returns the cluster assignment (s,t) of unit(s) (i,j) for i in I and j in J
    population size: n*n
    number of clusters: nc*nc

    Parameters
    -----------
    i : int or np.array
        row position of unit on n by n lattice (or array of row positions)
    j : int or np.array
        column position of unit on n by n lattice (or array of col positions)
    k : int
        typical cluster side length (each cluster is itself a k by k grid graph with k << n)
    q1 : int
        "origin" row position marking the END (inclusive) of first cluster
    q2 : int
        "origin" col position marking the END (inclusive) of first cluster
    divides : bool
        if k divides n, set to True

    Returns
    -----------
    s : int
        row position of the cluster on nc by nc lattice (or array of row positions)
    t : int
        column position of the cluster on nc by nc lattice (or array of col positions)
    '''
    if divides:
        s = np.floor(I/k)
        t = np.floor(J/k)
    else:
        s = np.ceil((I-q1)/k)
        t = np.ceil((J-q2)/k)

    return s.astype(int),t.astype(int)

def lat_toUnit(s,t,k,n,q1=0,q2=0):
    '''
    Returns the (row,column) indices of popluation units (on the n by n lattice) that belong to cluster (s,t)
    population size: n*n
    number of clusters: nc*nc

    Parameters
    -----------
    s (int): 
        row position of cluster on the nc by nc lattice of clusters
    t (int):
        column position of cluster on the nc by nc lattice of clusters
    k (int):
        typical cluster side length (average cluster contains k*k units)
    n (int): 
        population is represented by an n by n square lattice/grid graph
    q1 (int):
        "origin" row position marking the end (inclusive) of first cluster
    q2 (int):
        "origin" col position marking the end (inclusive) of first cluster
    
    Returns
    -------
    I (numpy array):
        TODO
    J (numpy array):
        TODO
    '''
    if n%k==0:
        starti = np.maximum(0,s*k)
        stopi = (s+1)*k - 1
        startj = np.maximum(0,t*k)
        stopj = (t+1)*k - 1
    else:
        starti = np.maximum(0,q1 + 1 + (s-1)*k)
        stopi = np.minimum(q1 + s*k, n-1)
        startj = np.maximum(0,q2 + 1 + (t-1)*k)
        stopj = np.minimum(q2 + t*k, n-1)
    
    I = np.linspace(starti,stopi,stopi-starti+1,dtype=int)
    J = np.linspace(startj,stopj,stopj-startj+1,dtype=int)
    
    return I,J

def cluster_neighborhood(A,i,k):
    '''
    Returns a list of tuples corresponding to the labels (s,t) of clusters adjacent to i
    A = adjacency matrix for population size n^2
    i = the unit we want to compute a neighborhood for
    k = "typical" cluster side length
    '''
    pop_size = np.shape(A)[0] 
    n = int(np.sqrt(pop_size))  # population size is n^2
    #nc = int(np.ceil(n/k)**2)   # number of clusters is nc^2

    # get indicies of i's neighbors (nonzero entries in i-th row of A)
    neighbors = np.nonzero(A[[i],:])[1]
    
    # We have nc^2 clusters represented by an nc x nc grid
    # We have labels (s,t) in [nc] x [nc] for each cluster
    # We also have labels k in [nc^2] for each cluster
    # Given (s,t), k = nc*s + t. Given k, (s,t)=(np.floor(k/nc),k%nc).
    # For each neighbor, get their cluster assignment (s,t)
    cluster_assignments = []
    for x in np.nditer(neighbors):
        # get the (i,j) coordinate of this neighbor on the population lattice [n] x [n]
        i = int(np.floor(x/n))
        j = x % n
        s,t = lat_toCluster(i,j,k,divides=(n%k==0))
        cluster_assignments.append((s,t))
    
    # remove duplicates
    cluster_assignments = list(set(cluster_assignments))

    return cluster_assignments

def bernoulli_cluster(num,p,clusters):
    '''
    num (int): number of clusters (should be a perfect square nc*nc)
    p (float): treatment probability in (0,1)
    clusters (N by 1 numpy array): clusters[i] = j says unit i in [N] is in cluster j in [NC]

    z (numpy array): i-th element is treatment assignment of unit i
    Cz (numpy array): (s,t)-th element is treatment assignment of cluster (s,t)
    flatCz (numpy array): given cluster label k in [nc*nc], return treatment assignment
    '''
    nc = int(np.sqrt(num))
    Cz = (np.random.rand(nc,nc) < p) + 0 # matrix where (s,t) entry is treatment assignment of cluster (s,t)
    flatCz = Cz.flatten() # each cluster (s,t) gets assigned to an index i in [c] where c = number of clusters = (rows+1)*(cols+1)
    treated_cluster_indices = np.where(flatCz == 1)[0] # returns the index labels of clusters that are assigned to treatment
    z = np.isin(clusters,treated_cluster_indices)+0 # if a person i is assigned to a treated cluster, then z(i) should equal 1 
    return Cz, flatCz, z

In [15]:
def select_clusters_bernoulli(numOfClusters, q):
    '''

    Parameters
    ------------
    numOfClusters : int
        NC=nc**2; the total number of clusters; given population size n*n, NC = ceil(n/k)**2 where k is the cluster side length
    q : float
        fraction of clusters you wish to select (in expectation)

    Returns
    --------
    selected : numpy array
        randomly selected clusters according to Bernoulli(q) design
        e.g. if you have clusters [0,1,2,3,4,5], this function independently tosses a coin with bias p for each cluster to determine if it will be selected
        and returns chosen = [1,5] if, for example, those were selected by the randomized design

    '''

    design = (np.random.rand(numOfClusters) < q) + 0
    selected = np.where(design == 1)
    return selected

## Testing

# Experiments

In [5]:
####### Construct Network ########

N = 8**2        # population size
n = 8

# Create lattice network
A = lattice2Dsq(n,n);

/Users/mayleencortez/Desktop/NetworkCausalInference/ClusterRD/low-order-clusterRD/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
k = np.array([1,2,4,8]) # 1x1, 2x2, 4x4, and 8x8 size clusters
for num in k:
    NC = int(np.ceil(n/num)**2) # number of clusters
    clusters = bf_clusters(NC,N)[0]
    #print(clusters)